In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib
import re
from datetime import datetime
from wordcloud import WordCloud 
import jieba
import jieba.analyse
from datetime import date
import re
import string
from wordcloud import WordCloud 

### 構面字典

In [13]:
# dict_ = pd.read_csv('../構面字典-new.csv')
dict_ = pd.read_csv('./構面字典-3.csv')
df_dict = dict_[['name','alias']]
d_list = df_dict.to_dict(orient='records')
structure_dict = {} # 構面字典 => name:alias
for i in d_list:
    structure_dict[i['name']] = i['alias']
structure_dict.keys()#key:成就感,...

dict_keys(['成就感', '學習成長', '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模', '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全', '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種', '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標', '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員', '正向', '負向'])

In [14]:
structure_origin = dict_[['class','name']]
structures_list = structure_origin['class'].unique().tolist() # 有哪些構面字陣列

structure_origin_list = pd.DataFrame(structure_origin.groupby('class')['name'].apply(list)).reset_index().to_dict(orient='records')
transformed_data = [{item['class']: item['name']} for item in structure_origin_list] # 取得構面字典，型態為list of dict

In [15]:
# 以pattern_text 例如是成就感，那就會去找text中有出現成就感的alias的次數
def get_dict_num(pattern_text, text):
    pattern = r"(" + structure_dict[pattern_text] + ")"
    num = len(re.findall(pattern, text))
    return num

In [47]:
# 取得資料
df_ = pd.read_csv('./salary_data.csv')
df_.drop('Unnamed: 0', axis=1,inplace=True)
df_ = df_[~df_['month_salary'].isna()].reset_index(drop='index')
df_.drop(['month_salary','year_salary','year-end_salary','bonus','dividends','other','average_day','seniority','total_seniority','mood'], axis = 1, inplace=True)
df_.head()

,vacancies,company,post_time,job,location,frequency_overtime,work_content,advise_content
0,生產技術工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.03.06,全職,苗栗,常常加班,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待\n1.因為要解析車子的問題但是因...,如果想進裕隆的朋友是非常喜歡寫報告及報告的人那這一間你會非常適合同一份報告，你可以寫三次分不...
1,工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.02.28,全職,苗栗,常常加班,報告很多，實用價值低\n會議很多，有開等於沒開，開完一堆追蹤\n對主要專精車輛事業不如中華汽...,想學報告製作的可以來\n抗壓、奴性強可來\n不在意工作樂趣可來\n不追求成就感可來\n想比傳...
2,品檢技術員,裕隆集團_裕隆汽車製造股份有限公司,2022.11.23,全職,苗栗,很少加班,組車零件進料檢出，關係企業供應商不好搞，常搬出集團的長官壓你，廠商送來零件沒有在出貨檢卻要你...,傳產薪資待遇，科技業不人性管理，除了苗栗三義、銅鑼附近的人推薦，其他要三思，學車廠經驗OK不...
3,生產技術工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.03.06,全職,苗栗,常常加班,公司算是老公司，所以做法非常老派，但是非常穩定可以穩穩的待\n1.因為要解析車子的問題但是因...,如果想進裕隆的朋友是非常喜歡寫報告及報告的人那這一間你會非常適合同一份報告，你可以寫三次分不...
4,工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.02.28,全職,苗栗,常常加班,報告很多，實用價值低\n會議很多，有開等於沒開，開完一堆追蹤\n對主要專精車輛事業不如中華汽...,想學報告製作的可以來\n抗壓、奴性強可來\n不在意工作樂趣可來\n不追求成就感可來\n想比傳...


In [18]:
# 字典型態的資料
# 將資料轉為字典，比較方便去做計算
dict_ = df_.to_dict(orient='records') 

In [19]:
# 把工作內容跟建議內容合併
for data in dict_:
    if(type(data['work_content'])==float):
        data['content'] = ''
    else:
        data['content'] = str(data['work_content'])+','+str(data['advise_content'])

In [20]:
# 取得根據每個構面下的類別名稱，計算該類別名稱出現多少次，來去計算dtm
for structure in list(structure_dict.keys()):
    for data in dict_:
        data[structure] = get_dict_num(structure, data['content'])

In [45]:
# DTM for 趨勢圖
# 把用不到的欄位踢除
# kick_off_key = ['vacancies','company','post_time','job','location','frequency_overtime','work_content','advise_content','content']
kick_off_key = ['job','location','frequency_overtime','work_content','advise_content','content']
data_list = [{k: v for k, v in d.items() if k not in kick_off_key} for d in dict_]
# 得到DTM DataFrame
dtm_df = pd.DataFrame(data_list)
dtm_df.head()

,vacancies,company,post_time,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,生產技術工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.03.06,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,3,3,203
1,工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.02.28,1,0,0,1,0,1,1,...,1,0,0,0,0,0,1,0,3,140
2,品檢技術員,裕隆集團_裕隆汽車製造股份有限公司,2022.11.23,1,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,1,183
3,生產技術工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.03.06,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,3,3,203
4,工程師,裕隆集團_裕隆汽車製造股份有限公司,2023.02.28,1,0,0,1,0,1,1,...,1,0,0,0,0,0,1,0,3,140


In [46]:
# DTM for correlation & co-ocurence #把用不到的欄位踢除
kick_off_ = ['vacancies','company','post_time','job','location','frequency_overtime','work_content','advise_content','content']
co_ = [{k: v for k, v in d.items() if k not in kick_off_} for d in dict_]
dtm_co = pd.DataFrame(co_)
dtm_co.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,0,0,0,0,0,0,1,0,3,0,...,0,0,0,0,0,0,0,3,3,203
1,1,0,0,1,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,3,140
2,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,183
3,0,0,0,0,0,0,1,0,3,0,...,0,0,0,0,0,0,0,3,3,203
4,1,0,0,1,0,1,1,0,0,0,...,1,0,0,0,0,0,1,0,3,140


### 計算correlation

In [38]:
df_correlation = pd.DataFrame(np.corrcoef(dtm_co.T),columns=dtm_co.columns, index=dtm_co.columns)

c:\Users\User\anaconda3\envs\tarnlp\lib\site-packages\numpy\lib\function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\User\anaconda3\envs\tarnlp\lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [51]:
df_corre_vect = df_correlation.reset_index()
df_corre_vect.rename(columns={'index': 'item1'}, inplace=True)
df_corre_vect.head()

,item1,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,成就感,1.000000,0.061157,-0.022705,0.063353,-0.126709,-0.199469,0.148439,-0.183507,0.076152,...,0.100328,-0.098096,NaN,-0.088691,-0.076401,-0.019653,-0.145210,0.085325,0.125423,0.264424
1,學習成長,0.061157,1.000000,-0.000517,-0.261352,-0.104154,-0.095633,-0.125592,0.228092,-0.215013,...,0.036593,0.026125,NaN,-0.161000,0.116317,-0.035676,0.345660,-0.198111,-0.149955,-0.170746
2,創新,-0.022705,-0.000517,1.000000,0.016977,0.146301,-0.033192,-0.039957,-0.032369,-0.008563,...,-0.011053,-0.017303,NaN,-0.015645,-0.013477,-0.003467,0.011267,-0.006929,0.060026,0.103935
3,薪資,0.063353,-0.261352,0.016977,1.000000,0.367595,0.227468,-0.036655,0.027380,-0.076963,...,0.007973,-0.159546,NaN,0.091043,0.198511,0.135234,-0.188385,-0.116390,0.002599,-0.019937
4,福利,-0.126709,-0.104154,0.146301,0.367595,1.000000,0.309386,0.051686,-0.080539,0.021644,...,-0.046056,0.046105,NaN,0.008001,0.197394,0.126637,-0.178164,-0.023915,0.023977,0.108651


In [40]:
corr_melt_df = pd.melt(df_corre_vect, id_vars='item1', value_vars=list(df_corre_vect.columns)[1:], var_name='item2', value_name='correlation')
corr_melt_df.rename(columns={'index':'item1'},inplace=True)
corr_melt_df

,item1,item2,correlation
0,成就感,成就感,1.000000
1,學習成長,成就感,0.061157
2,創新,成就感,-0.022705
3,薪資,成就感,0.063353
4,福利,成就感,-0.126709
...,...,...,...
3244,出路,負向,0.072542
3245,能力,負向,-0.125754
3246,招募人員,負向,0.579410
3247,正向,負向,0.084140


In [43]:
corr_melt_df[corr_melt_df['item1'] == '負向'].sort_values(['correlation'], ascending=False)[1:7]

,item1,item2,correlation
1025,負向,馬力,0.999694
3134,負向,招募人員,0.579410
512,負向,主管風格,0.563723
56,負向,成就感,0.264424
2792,負向,離職,0.232919
740,負向,產業前景,0.231797


### 計算 co-occurrence

In [52]:
result = np.dot(dtm_co.T.values, dtm_co.values)
co_occurrence_df = pd.DataFrame(result, columns=dtm_co.columns, index=dtm_co.columns)
co_occurrence_df.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
成就感,306,138,0,198,45,102,165,0,180,45,...,324,0,0,0,0,0,51,180,726,54132
學習成長,138,1151,4,265,229,564,217,389,212,61,...,849,54,0,0,54,0,990,212,1529,119684
創新,0,4,12,8,16,4,0,0,4,0,...,8,0,0,0,0,0,8,4,36,2540
薪資,198,265,8,1869,751,1217,434,367,547,154,...,1216,21,0,84,93,12,475,476,2732,196524
福利,45,229,16,751,870,819,309,155,394,93,...,665,52,0,36,62,8,247,340,1648,124125


In [48]:
df_ocurr_vect = co_occurrence_df.reset_index()
df_ocurr_vect.rename(columns={'index': 'item1'}, inplace=True)
df_ocurr_vect
ocurr_melt_df = pd.melt(df_ocurr_vect, id_vars='item1', value_vars=list(df_ocurr_vect.columns)[1:], var_name='item2', value_name='ocurrelation')
ocurr_melt_df.rename(columns={'index':'item1'},inplace=True)
ocurr_melt_df

,item1,item2,ocurrelation
0,成就感,成就感,306
1,學習成長,成就感,138
2,創新,成就感,0
3,薪資,成就感,198
4,福利,成就感,45
...,...,...,...
3244,出路,負向,1176
3245,能力,負向,184777
3246,招募人員,負向,213622
3247,正向,負向,722949
